This notebook reads in netCDF inflow plane data that has been dimensioned properly, and it output time-blocks of numpy data that is organized by campaign number.

In [1]:
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
from matplotlib.colors import TwoSlopeNorm
import matplotlib.dates as mdates
from pathlib import Path
import pandas as pd
import scipy.interpolate as interpolate

import warnings
warnings.simplefilter("ignore") 

In [2]:
### User inputs
n_tsteps = 256
nvars_out = 5  # velocityx, velocityy, velocityz, temperature, tke
plane_loc = '3D'
outdir = Path(f'/scratch/orybchuk/wakedynamics/bcs-ldm/data/072415/post_processing/inflow-bc-{plane_loc}/npy_blocks{n_tsteps}')
outdir.mkdir(exist_ok=True, parents=True)

downsample_time = True
bc_out_tinterval = 1.0  # sec, the save frequency that we downsample to

In [3]:
### Read in data
parent_dir = Path('/scratch/orybchuk/wakedynamics/bcs-ldm/data/072415/post_processing/nc_dimensioned')

bc_files = list(parent_dir.glob(f'inflow-bc-{plane_loc}*.nc'))
bc_files.sort()
n_campaigns = len(bc_files)

# campaign_num = str(0).zfill(4)
# ds_bc = xr.open_dataset(Path(parent_dir,f'inflow-bc{campaign_num}.nc'))

In [4]:
### Prepare to convert the files
var_maxes = np.zeros(nvars_out)
var_mins = np.zeros(nvars_out)

In [5]:
### Convert to blocked numpy files
for icamp, fcampaign in enumerate(bc_files):  # Iterate over campaigns
    if icamp % 10 == 0: print(datetime.now(), icamp)
    ds_bc = xr.open_dataset(fcampaign)
    
    if downsample_time:  # Optionally downsample the BC data
        bc_orig_tinterval = ds_bc['time'].values[1] - ds_bc['time'].values[0]  # sec, the original save frequency
        assert bc_out_tinterval%bc_orig_tinterval==0.0, "bc_out_tinterval needs to be a multiple of bc_orig_tinterval!"
        coarse_time = np.arange(ds_bc['time'].values[0], ds_bc['time'].values[-1], bc_out_tinterval)
        ds_bc = ds_bc.sel(time=coarse_time)
        
    n_full_windows = np.floor(len(ds_bc['time']) / n_tsteps).astype(int)
    
    for iwind in range(n_full_windows):  # Iterate over time windows
        ds_trim = ds_bc.isel(x=0, time=slice(iwind*n_tsteps, (iwind+1)*n_tsteps))

        
        ### ----- Save out data -----
        arr_out = np.zeros((nvars_out, n_tsteps, len(ds_bc['y']), len(ds_bc['z'])))
        arr_out[0,:,:,:] = ds_trim['velocityx'].values
        arr_out[1,:,:,:] = ds_trim['velocityy'].values
        arr_out[2,:,:,:] = ds_trim['velocityz'].values
        arr_out[3,:,:,:] = ds_trim['temperature'].values
        arr_out[4,:,:,:] = ds_trim['tke'].values
        
        np.save(Path(outdir, f'{str(icamp).zfill(4)}_{str(iwind).zfill(3)}.npy'), arr_out)
        
        ### ----- Update statistics -----
        if icamp == 0:
            for j in range(nvars_out):
                var_maxes[j] = arr_out[j,:,:,:].max()
                var_mins[j] = arr_out[j,:,:,:].min()
        else:
            for j in range(nvars_out):
                var_maxes[j] = max(var_maxes[j], arr_out[j,:,:,:].max())
                var_mins[j] = min(var_mins[j], arr_out[j,:,:,:].min())
                
## Save statistics
with open(Path(outdir, "stats.txt"), 'w') as fstats:
    for i in range(nvars_out):
        fstats.write(f"Variable number {i} max: {var_maxes[i]}\n")
        fstats.write(f"Variable number {i} min: {var_mins[i]}\n")
    
print("Variable maximums:", var_maxes)
print("Variable minimums:", var_mins)

2024-02-18 11:42:43.086240 0
2024-02-18 11:49:40.143580 10
2024-02-18 11:57:33.326587 20
2024-02-18 12:05:34.038573 30
2024-02-18 12:12:14.494062 40
2024-02-18 12:21:40.878394 50
2024-02-18 12:29:11.380624 60
2024-02-18 12:37:20.217679 70
2024-02-18 12:43:59.606869 80
2024-02-18 12:51:38.587348 90
2024-02-18 12:59:19.677076 100
2024-02-18 13:07:39.412737 110
2024-02-18 13:15:32.028750 120
2024-02-18 13:23:34.523924 130
2024-02-18 13:30:56.829188 140
2024-02-18 13:39:24.899674 150
2024-02-18 13:47:11.364501 160
2024-02-18 13:55:28.435479 170
2024-02-18 14:03:09.257153 180
2024-02-18 14:10:55.201411 190
2024-02-18 14:19:14.970072 200
2024-02-18 14:27:55.696563 210
2024-02-18 14:35:25.316008 220
2024-02-18 14:42:35.212492 230
2024-02-18 14:49:56.925832 240
2024-02-18 14:58:16.093126 250
2024-02-18 15:06:40.036231 260
2024-02-18 15:14:49.376145 270
2024-02-18 15:23:37.235061 280
2024-02-18 15:30:48.187373 290
2024-02-18 15:38:07.431421 300
2024-02-18 15:51:49.854395 310
2024-02-18 15:58:50